In [28]:
import itertools
import numpy as np
import pandas as pd

In [18]:
data = pd.read_json('./data/jobs.json', orient='records')
data.job_description = data.job_description.apply(lambda t: t[0])
data

,date,job_description,salary,skills,title,url
0,2017-02-10,"<div class=""description"" itemprop=""description...","£55,000 per annum","[Oracle, SSRS, DBA, SQL Server]",Database Administrator,https://www.reed.co.uk/jobs/database-administr...
1,2017-02-09,"<div class=""description"" itemprop=""description...","£45,000 - £50,000 per annum",[],IT Manager - Clinical systems experience,https://www.reed.co.uk/jobs/it-manager-clinica...
2,2017-02-09,"<div class=""description"" itemprop=""description...",£250.00 - £300.00 per day,[],Senior Web / WinForms Developer - ASP.Net / C...,https://www.reed.co.uk/jobs/senior-web-winform...
3,2017-02-09,"<div class=""description"" itemprop=""description...","£35,000 - £45,000 per annum",[],JavaScript Developer (Angular),https://www.reed.co.uk/jobs/javascript-develop...
4,2017-02-09,"<div class=""description"" itemprop=""description...","£30,000 - £40,000 per annum","[ASP.NET, C#, MVC, Sitecore, Episerver]",Software Developer,https://www.reed.co.uk/jobs/software-developer...
5,2017-02-09,"<div class=""description"" itemprop=""description...",£290.00 - £300.00 per day,"[Oracle, Scrum, Testing, ISEB, STLC, agile, da...",Testers,https://www.reed.co.uk/jobs/testers/31626707
6,2017-02-09,"<div class=""description"" itemprop=""description...",£350.00 - £360.00 per day,"[Multitasking, QA and Testing, Meeting project...",Test Managers,https://www.reed.co.uk/jobs/test-managers/3162...
7,2017-02-09,"<div class=""description"" itemprop=""description...",£350.00 - £360.00 per day,"[Multitasking, QA and Testing, Meeting project...",Test Manager,https://www.reed.co.uk/jobs/test-manager/31626747
8,2017-02-09,"<div class=""description"" itemprop=""description...","£45,000 - £55,000 per annum","[Linux, Redis, AWS, Vagrant, DevOps, Laravel, ...",DevOps,https://www.reed.co.uk/jobs/devops/31626762
9,2017-02-09,"<div class=""description"" itemprop=""description...","£65,000 - £70,000 per annum, negotiable, pro-r...","[Applications, Project Management, Technology]",BI Project Manager,https://www.reed.co.uk/jobs/bi-project-manager...


# Term-Document Matrix

In [30]:
all_skills = data.skills.tolist()
all_skills = np.asarray(all_skills)


all_skills = list(itertools.chain(*all_skills))
all_skills[:5]

[u'Oracle', u'SSRS', u'DBA', u'SQL Server', u'ASP.NET']

In [37]:
td_matrix = pd.DataFrame()
td_matrix['skills'] = all_skills
td_matrix[:5]

,skills
0,Oracle
1,SSRS
2,DBA
3,SQL Server
4,ASP.NET
